IMDB 영화평데이터 > 감성분류를 위한 트랜스포머 아키텍처 모델 구축

1. 정수토큰 시퀀스(길이80)입력
2. 토큰임베딩 + 위치임베딩
3. 멀티헤드어텐션 3헤드
4. concate+정규화
5. FFN (Dense+Dense)
6. concat+정규화
7. 분류기 (Dense)

# 1. 정수토큰 시퀀스(길이80)입력

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers

2025-09-05 11:48:39.724966: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# 토큰 임베딩
inputs = layers.Input(shape=(80,))
input_embedding = layers.Embedding(input_dim=1000, output_dim=32)(inputs)

I0000 00:00:1757040664.628812  201308 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4459 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


# 2. 토큰임베딩 + 위치임베딩

In [3]:
# 위치임베딩
positions = tf.range(start=0, limit=80)
pos_embedding = layers.Embedding(input_dim=80, output_dim=32)(positions)
pos_enc_output = pos_embedding + input_embedding

# 3. 멀티헤드어텐션 3헤드

In [ ]:
attention_output = layers.MultiHeadAttention(num_heads=3, key_dim=32)(pos_enc_output, pos_enc_output) #K,V

# 4. concate+정규화

In [6]:
x = layers.add([pos_enc_output, attention_output])
x = layers.BatchNormalization()(x)

# 5. FFN (Dense+Dense)
# 6. concat+정규화

In [8]:
from tensorflow.keras.models import Sequential
ffnn = Sequential(
  [
    layers.Dense(64,activation='relu'),
    layers.Dense(32, activation='relu')
  ]
)(x)
x = layers.add([ffnn, x])
x = layers.BatchNormalization()(x)

# 7. 분류기 (Dense)